In [ ]:
import cv2
import pytesseract
import json
from pytesseract import Output as PytesseractOutput
from IPython.display import display
from ipyfilechooser import FileChooser
from ipywidgets import Button, Output

# Function to perform OCR on an image
def perform_ocr(image_path):
    # Read image using OpenCV
    image = cv2.imread(image_path)
    
    # Pre-process the image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_image = cv2.medianBlur(gray_image, 3)  # Remove noise
    _, binary_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)  # Binarization
    
    # Perform OCR using Tesseract with custom configuration
    custom_config = r'--oem 3 --psm 11'
    d = pytesseract.image_to_data(binary_image, config=custom_config, output_type=PytesseractOutput.DICT)
    
    # Extract recognized text and bounding boxes
    n_boxes = len(d['text'])
    ocr_results = []
    for i in range(n_boxes):
        if int(d['conf'][i]) > 50:  # Consider text only if confidence is above 60
            text = d['text'][i]
            x, y, w, h = d['left'][i], d['top'][i], d['width'][i], d['height'][i]
            ocr_results.append({
                "text": text,
                "bounding_box": [x, y, w, h]
            })
            # Draw bounding box on the image
            image = cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            # Put text on the image
            image = cv2.putText(image, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)
    
    return ocr_results, image

# Function to process multiple images and save results to a JSON file
def process_images(image_paths, output_file):
    results = {}
    for image_path in image_paths:
        ocr_results, annotated_image = perform_ocr(image_path)
        results[image_path] = ocr_results
        # Save the image with bounding boxes and text
        output_image_path = image_path.replace('.jpg', '_annotated.jpg')
        cv2.imwrite(output_image_path, annotated_image)
    
    # Save results to a JSON file
    with open(output_file, 'w') as outfile:
        json.dump(results, outfile, indent=4)

# Create and display a FileChooser widget
fc = FileChooser('../')
# Set a file filter pattern
fc.filter_pattern = '*.jpg'
display(fc)

# Output widget to capture print statements
output = Output()

# Function to get selected image paths from FileChooser
def get_selected_images(fc):
    return [fc.selected]

# Button to start processing
button = Button(description="Start Processing")
display(button)

# Output JSON file
output_file = 'ocr_results.json'

# Function to be called when button is clicked
def on_button_clicked(b):
    with output:
        selected_images = get_selected_images(fc)
        if selected_images and selected_images[0]:
            process_images(selected_images, output_file)
            print("Processing complete. Results saved to", output_file)
        else:
            print("No images selected.")
            
button.on_click(on_button_clicked)
display(output)
